In [2]:
from experta import *

class States(Fact):
    @Rule()
    def Ask_Vegetation():
        self.declare(Fact())


engine = States()
engine.reset()
engine.run()

AttributeError: 'States' object has no attribute 'reset'